In [ ]:
!pip install "crewai[tools]" google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of instructor to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 kB 18.3

## Project : Resume & Job Description Matcher Crew


Input:

1. resume_text (candidate’s resume)

2. jd_text (job description)

3. Optional: job_title, company_name

Agents & tasks (sequential):

1. ResumeExtractorAgent → structured summary of candidate

2. JDExtractorAgent → structured summary of job

3. MatcherAgent → match score + skill overlap + gaps

4. ResumeImproverAgent → JD-aligned bullets for resume

5. CoverLetterAgent → tailored cover letter



In [ ]:
# Get API keys from environment variables
from google.colab import userdata
import google.generativeai as genai
import os

GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GEMINI_API_KEY:
    print("Warning: GEMINI_API_KEY not found. Gemini model will not run.")

genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
import os
import google.generativeai as genai
from crewai import LLM


#  Create Gemini-backed LLM for CrewAI
gemini_model = "gemini-2.5-flash"
gemini_llm = LLM(
    model=gemini_model,
    temperature=0.1,
    api_key=GEMINI_API_KEY
)


## 2️ Define Agents

In [ ]:
from crewai import Agent

resume_extractor = Agent(
    role="Resume Extraction Specialist",
    goal=(
        "Read a candidate's resume and extract structured information such as "
        "skills, tools, experience, domains, and key achievements."
    ),
    backstory=(
        "You work in HR analytics and ATS systems, and you are excellent at "
        "turning messy resumes into clean, structured profiles."
    ),
    llm=gemini_llm,
    verbose=True,
    allow_delegation=False,
)

jd_extractor = Agent(
    role="Job Description Analyst",
    goal=(
        "Analyze a job description and extract required skills, nice-to-have "
        "skills, responsibilities, and experience requirements."
    ),
    backstory=(
        "You are a recruiter who deeply understands how to interpret job posts "
        "into clear hiring criteria."
    ),
    llm=gemini_llm,
    verbose=True,
    allow_delegation=False,
)

matcher_agent = Agent(
    role="Resume-JD Matching Analyst",
    goal=(
        "Compare a candidate profile with a job description and compute a match "
        "score, overlapping skills, and skill gaps."
    ),
    backstory=(
        "You help recruiters quickly see how well a candidate fits a role, "
        "explaining strengths and gaps clearly."
    ),
    llm=gemini_llm,
    verbose=True,
    allow_delegation=False,
)

resume_improver = Agent(
    role="Resume Improvement Specialist",
    goal=(
        "Improve the candidate's resume content so that it better matches the "
        "job description while staying honest and accurate."
    ),
    backstory=(
        "You specialize in rewriting bullet points and summaries to highlight "
        "relevant experience and achievements for a specific job."
    ),
    llm=gemini_llm,
    verbose=True,
    allow_delegation=False,
)

cover_letter_writer = Agent(
    role="Cover Letter Writer",
    goal=(
        "Write a tailored, professional cover letter for a specific job and "
        "company based on the candidate's profile and the job description."
    ),
    backstory=(
        "You are an experienced career coach and writer who crafts impactful cover "
        "letters that clearly show fit and motivation."
    ),
    llm=gemini_llm,
    verbose=True,
    allow_delegation=False,
)


## Define Tasks (with both resume & JD)

In [ ]:
from crewai import Task

def build_resume_jd_tasks(resume_text: str,
                          jd_text: str,
                          job_title: str,
                          company_name: str):
    # 1) Resume extraction
    task_resume_extract = Task(
        description=(
            "You are given a candidate's resume. Extract structured information.\n\n"
            "Resume:\n"
            f"{resume_text}\n\n"
            "Your output MUST follow this structured markdown format:\n\n"
            "## Candidate Summary\n"
            "- Short 2–3 sentence summary of the candidate.\n\n"
            "## Skills\n"
            "- List of main technical skills\n"
            "- List of main soft skills\n\n"
            "## Tools & Technologies\n"
            "- List of tools, libraries, frameworks, platforms\n\n"
            "## Experience\n"
            "- Estimated total years of experience\n"
            "- Bullet points of major roles or projects\n\n"
            "## Domains / Industries\n"
            "- List any industries or domains the candidate has worked in\n"
        ),
        expected_output=(
            "A structured markdown profile of the candidate with sections: "
            "Candidate Summary, Skills, Tools & Technologies, Experience, Domains/Industries."
        ),
        agent=resume_extractor,
    )

    # 2) JD extraction
    task_jd_extract = Task(
        description=(
            "You are given a job description. Extract structured hiring criteria.\n\n"
            f"Job Title: {job_title}\n"
            f"Company: {company_name}\n\n"
            "Job Description:\n"
            f"{jd_text}\n\n"
            "Your output MUST follow this structured markdown format:\n\n"
            "## Role Summary\n"
            "- 2–3 sentence summary of what this role is about\n\n"
            "## Required Skills\n"
            "- Bullet list of must-have skills\n\n"
            "## Nice-to-have Skills\n"
            "- Bullet list of optional or bonus skills\n\n"
            "## Experience Requirements\n"
            "- Years of experience\n"
            "- Type of experience expected (e.g., backend, data, product)\n\n"
            "## Key Responsibilities\n"
            "- Bullet list of main responsibilities\n"
        ),
        expected_output=(
            "A structured markdown analysis of the job with sections: Role Summary, "
            "Required Skills, Nice-to-have Skills, Experience Requirements, Key Responsibilities."
        ),
        agent=jd_extractor,
    )

    # 3) Matching
    task_matching = Task(
        description=(
            "You have access to:\n"
            "- A structured candidate profile\n"
            "- A structured job description analysis\n\n"
            "Compare the candidate with the job and produce a clear matching analysis.\n\n"
            "Your output MUST follow this structured markdown format:\n\n"
            "## Match Score\n"
            "- Overall match percentage (0–100) with a one-line explanation\n\n"
            "## Strong Matches\n"
            "- Bullet list of skills/experience where the candidate matches or exceeds requirements\n\n"
            "## Partial Matches\n"
            "- Bullet list of areas where the candidate is somewhat aligned but not perfect\n\n"
            "## Gaps / Missing Skills\n"
            "- Bullet list of missing or weak skills compared to the JD\n\n"
            "## Recommendations\n"
            "- Suggestions for how the candidate could improve their fit for this role\n"
        ),
        expected_output=(
            "A structured matching report with sections: Match Score, Strong Matches, "
            "Partial Matches, Gaps / Missing Skills, Recommendations."
        ),
        agent=matcher_agent,
    )

    # 4) Resume improvement
    task_resume_improve = Task(
        description=(
            "Using the original resume, the candidate profile, the job analysis, and "
            "the matching report, propose improved resume content focused on this job.\n\n"
            "Job Title: {job_title}\n"
            "Company: {company_name}\n\n"
            "Original resume text is available in context. Focus on:\n"
            "- Rewriting or adding 5–10 bullet points that better highlight relevant experience\n"
            "- A short 2–3 line professional summary tailored to this role\n\n"
            "Your output MUST follow this structured markdown format:\n\n"
            "## Tailored Professional Summary\n"
            "- 2–3 line summary\n\n"
            "## Recommended Bullet Points\n"
            "- Bullet points suitable for an Experience or Projects section\n"
        ).format(job_title=job_title, company_name=company_name),
        expected_output=(
            "Tailored professional summary and recommended bullet points for the resume."
        ),
        agent=resume_improver,
    )

    # 5) Cover letter
    task_cover_letter = Task(
        description=(
            "Write a tailored cover letter for this candidate applying to the given role.\n\n"
            f"Job Title: {job_title}\n"
            f"Company: {company_name}\n\n"
            "Use the candidate's strengths, the matching analysis, and the improved resume "
            "bullets to write a convincing but honest cover letter.\n\n"
            "Constraints:\n"
            "- Length: 3–6 short paragraphs\n"
            "- Professional but warm tone\n"
            "- Mention the company and role by name\n"
            "- Highlight the strongest matches and address any minor gaps positively\n"
        ),
        expected_output="A complete, polished cover letter ready to send.",
        agent=cover_letter_writer,
    )

    return [
        task_resume_extract,
        task_jd_extract,
        task_matching,
        task_resume_improve,
        task_cover_letter,
    ]


## 4️ Build the Crew and Run Everything

In [ ]:
from crewai import Crew, Process

def run_resume_jd_matching(resume_text: str,
                           jd_text: str,
                           job_title: str = "Software Engineer",
                           company_name: str = "Example Corp"):
    tasks = build_resume_jd_tasks(
        resume_text=resume_text,
        jd_text=jd_text,
        job_title=job_title,
        company_name=company_name,
    )

    crew = Crew(
        agents=[
            resume_extractor,
            jd_extractor,
            matcher_agent,
            resume_improver,
            cover_letter_writer,
        ],
        tasks=tasks,
        process=Process.sequential,  # run tasks in order
        verbose=True,
    )

    # This will return the final task's output (cover letter) as 'result'
    result = crew.kickoff()
    return result


In [ ]:
sample_resume = """
John Doe
Software Engineer with 3+ years of experience in backend development.
Skills: Python, Django, REST APIs, PostgreSQL, Docker, Git, unit testing.
Experience:
- Built and maintained REST APIs for an e-commerce platform used by thousands of users daily.
- Collaborated with frontend team to design API contracts and improve performance.
- Deployed services using Docker and basic CI/CD pipelines.
Projects:
- Personal project: task management app using Django and React.
"""

sample_jd = """
We are looking for a Backend Software Engineer to join our platform team.

Responsibilities:
- Design, build, and maintain scalable backend services.
- Develop and document RESTful APIs.
- Work closely with frontend and product teams.
- Optimize performance and ensure reliability.

Requirements:
- 3-5 years of experience in backend development.
- Strong experience with Python or similar language.
- Experience with relational databases (preferably PostgreSQL or MySQL).
- Familiarity with Docker and CI/CD is a plus.
- Good communication and teamwork skills.
"""

final_cover_letter = run_resume_jd_matching(
    resume_text=sample_resume,
    jd_text=sample_jd,
    job_title="Backend Software Engineer",
    company_name="Acme Tech"
)

print("\n================ FINAL TAILORED COVER LETTER ================\n")
print(final_cover_letter)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 76373d9c-6d69-4174-b782-641aa45df1b0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resume Extraction Specialist                                                                            │
│                                                                                                                 │
│  Task: You are given a candidate's resume. Extract structured information.                                      │
│                                                                                                                 │
│  Resume:                                                                                                        │
│                                                                                                                 │
│  John Doe                                                                                                       │
│  Software Engineer with 3+ years of experience in backend development.                                          │
│  Skills: Python, Django, REST APIs, PostgreSQL, Docker, Git, unit testing.                                      │
│  Experience:                                                                                                    │
│  - Built and maintained REST APIs for an e-commerce platform used by thousands of users daily.                  │
│  - Collaborated with frontend team to design API contracts and improve performance.                             │
│  - Deployed services using Docker and basic CI/CD pipelines.                                                    │
│  Projects:                                                                                                      │
│  - Personal project: task management app using Django and React.                                                │
│                                                                                                                 │
│                                                                                                                 │
│  Your output MUST follow this structured markdown format:                                                       │
│                                                                                                                 │
│  ## Candidate Summary                                                                                           │
│  - Short 2–3 sentence summary of the candidate.                                                                 │
│                                                                                                                 │
│  ## Skills                                                                                                      │
│  - List of main technical skills                                                                                │
│  - List of main soft skills                                                                                     │
│                                                                                                                 │
│  ## Tools & Technologies                                                                                        │
│  - List of tools, libraries, frameworks, platforms                                                              │
│                                                                                                                 │
│  ## Experience                                                                                                  │
│  - Estimated total years of experience                                                                          │
│  - Bullet points of major roles or projects                                                                     │
│                                                        

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resume Extraction Specialist                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Candidate Summary                                                                                           │
│  - John Doe is a Software Engineer with over 3 years of experience specializing in backend development. He has  │
│  a proven track record of building and maintaining REST APIs for high-traffic e-commerce platforms.             │
│  - He is proficient in Python, Django, PostgreSQL, and Docker, and has experience collaborating with frontend   │
│  teams and deploying services using CI/CD pipelines.                                                            │
│                                                                                                                 │
│  ## Skills                                                                                                      │
│  - List of main technical skills                                                                                │
│      - Backend Development                                                                                      │
│      - REST API Design & Development                                                                            │
│      - Unit Testing                                                                                             │
│      - Database Management                                                                                      │
│      - CI/CD                                                                                                    │
│  - List of main soft skills                                                                                     │
│      - Collaboration                                                                                            │
│      - Problem-solving                                                                                          │
│                                                                                                                 │
│  ## Tools & Technologies                                                                                        │
│  - Python                                                                                                       │
│  - Django                                                                                                       │
│  - PostgreSQL                                                                                                   │
│  - Docker                                                                                                       │
│  - Git                                                                                                          │
│  - React (from personal project)                                                                                │
│                                                                                                                 │
│  ## Experience                                                                                                  │
│  - Estimated total years of experience                                                                          │
│      - 3+ years                                                                                                 │
│  - Bullet points of major roles or projects                                                                     │
│      - Built and maintained REST APIs for an e-commerce

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Cover Letter Writer                                                                                     │
│                                                                                                                 │
│  Task: Write a tailored cover letter for this candidate applying to the given role.                             │
│                                                                                                                 │
│  Job Title: Backend Software Engineer                                                                           │
│  Company: Acme Tech                                                                                             │
│                                                                                                                 │
│  Use the candidate's strengths, the matching analysis, and the improved resume bullets to write a convincing    │
│  but honest cover letter.                                                                                       │
│                                                                                                                 │
│  Constraints:                                                                                                   │
│  - Length: 3–6 short paragraphs                                                                                 │
│  - Professional but warm tone                                                                                   │
│  - Mention the company and role by name                                                                         │
│  - Highlight the strongest matches and address any minor gaps positively                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Cover Letter Writer                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  [Your Name]                                                                                                    │
│  [Your Address]                                                                                                 │
│  [Your Phone Number]                                                                                            │
│  [Your Email]                                                                                                   │
│                                                                                                                 │
│  [Date]                                                                                                         │
│                                                                                                                 │
│  Hiring Manager                                                                                                 │
│  Acme Tech                                                                                                      │
│  [Company Address - if known, otherwise omit]                                                                   │
│                                                                                                                 │
│  Dear Hiring Manager,                                                                                           │
│                                                                                                                 │
│  I am writing to express my enthusiastic interest in the Backend Software Engineer position at Acme Tech, as    │
│  advertised on [Platform where you saw the advertisement, e.g., LinkedIn]. With over three years of dedicated   │
│  experience in backend development, specializing in designing, building, and maintaining scalable RESTful       │
│  APIs, I am confident that my skills and proven track record align perfectly with the requirements of this      │
│  role.                                                                                                          │
│                                                                                                                 │
│  My background includes extensive work with Python and Django, where I have designed, built, and maintained     │
│  highly scalable RESTful APIs and backend services for high-traffic e-commerce platforms, supporting thousands  │
│  of daily users. I have a strong command of PostgreSQL for database management and have consistently optimized  │
│  backend service performance, ensuring reliability and high availability for critical applications.             │
│  Furthermore, my proficiency extends to leveraging Docker and implementing CI/CD pipelines to streamline        │
│  service deployment and enhance operational efficiency.                                                         │
│                                                                                                                 │
│  Beyond technical expertise, I thrive in collaborative environments. I have a proven ability to work closely    │
│  with frontend developers and product teams to define API specifications, develop and document comprehensive    │
│  RESTful APIs, and facilitate seamless integration. This collaborative approach, coupled with my                │
│  problem-solving skills, ensures that I can contribute 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0425be8d-8594-43a5-8609-7ce80f8ce5af                                                                     │
│  Agent: Cover Letter Writer                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

================ FINAL TAILORED COVER LETTER ================